<a href="https://colab.research.google.com/github/sariyumadagoni/advancedatabase/blob/main/neon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install SQLAlchemy psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.8 MB/s eta 0:00:00


In [1]:
import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: Top Five Most Expensive Products
with engine.connect() as conn:
    expensive_sql = """
    SELECT p.maker, pr.model, pr.price
    FROM product p
    JOIN (
        SELECT model, price FROM pc
        UNION ALL
        SELECT model, price FROM laptop
        UNION ALL
        SELECT model, price FROM printer
    ) pr USING (model)
    ORDER BY pr.price DESC
    LIMIT 5;
    """
    expensive_df = pd.read_sql_query(text(expensive_sql), conn)

# Display results
print("Top Five Most Expensive Products:")
display(expensive_df)

Paste STUDENT DB URL (hidden): ··········
Top Five Most Expensive Products:


,maker,model,price
0,E,2001,3673
1,A,2005,2500
2,G,2010,2300
3,A,1001,2114
4,A,2006,1700


In [2]:
# Query 2: Count of Products by Category
with engine.connect() as conn:
    count_sql = """
    SELECT type AS category, COUNT(*) AS product_count
    FROM product
    GROUP BY type
    ORDER BY product_count DESC;
    """
    category_counts_df = pd.read_sql_query(text(count_sql), conn)

# Display results
print(" Count of Products by Category:")
display(category_counts_df)

 Count of Products by Category:


,category,product_count
0,pc,13
1,laptop,10
2,printer,7


In [15]:
# Query 3: Average Price of In-Stock Items
with engine.connect() as conn:
    avg_price_sql = """
    SELECT ROUND(AVG(price), 2) AS avg_price
    FROM (
        SELECT price FROM pc WHERE price IS NOT NULL
        UNION ALL
        SELECT price FROM laptop WHERE price IS NOT NULL
        UNION ALL
        SELECT price FROM printer WHERE price IS NOT NULL
    ) all_items;
    """
    avg_price_df = pd.read_sql_query(text(avg_price_sql), conn)

# Display results
print("Average Price of In-Stock Items:")
display(avg_price_df)

Average Price of In-Stock Items:


,avg_price
0,945.3
